In [1]:
from __future__ import print_function
import keras
from keras.models import Sequential, Model, load_model

import tensorflow as tf

import pandas as pd

import os
import pickle
import numpy as np

import scipy.sparse as sp
import scipy.io as spio

import isolearn.io as isoio
import isolearn.keras as iso

from aparent.data.aparent_data_plasmid_legacy import load_data


Using TensorFlow backend.


In [2]:
#Load random MPRA data

file_path = '../../data/random_mpra_legacy/combined_library/processed_data_lifted/'
plasmid_gens = load_data(batch_size=32, valid_set_size=0.025, test_set_size=0.0375, file_path=file_path)


Training set size = 3115678
Validation set size = 83084
Test set size = 124627


In [3]:
#Load legacy APARENT model (lifted from theano)

model_name = 'aparent_theano_legacy_30_31_34_pasaligned'

save_dir = os.path.join(os.getcwd(), '../../saved_models/legacy_models')
model_path = os.path.join(save_dir, model_name + '.h5')

aparent_model = load_model(model_path)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


/home/johli/anaconda3/envs/aparent/lib/python3.6/site-packages/keras/engine/saving.py:292: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [4]:
#Predict from test data generator

iso_pred_test, cut_pred_test = aparent_model.predict_generator(plasmid_gens['test'], workers=4, use_multiprocessing=True)

#Calculate isoform logits
logodds_pred_test = np.ravel(np.log(iso_pred_test / (1. - iso_pred_test)))


In [5]:
#Copy the test set dataframe and store isoform predictions

plasmid_df = plasmid_gens['test'].sources['df']
test_df = plasmid_df.iloc[plasmid_gens['test'].data_ids[plasmid_gens['test'].indexes][:cut_pred_test.shape[0]]].copy().reset_index()

test_df['iso_pred'] = iso_pred_test
test_df['logodds_pred'] = logodds_pred_test

test_df = test_df[['seq', 'iso_pred', 'logodds_pred']]


In [6]:
#Dump prediction dataframe and cut probability matrix

isoio.dump({'plasmid_df' : test_df, 'cut_prob' : sp.csr_matrix(cut_pred_test)}, 'apa_plasmid_data_legacy/' + model_name + '_test_predictions')